## Installing tensorflow in your vm

You can pretty easily add tensorflow to your vm to support some small deep learning projects.  Follow the instructions below.

First open a terminal in your vm and do 

```
sudo apt-get update
sudo apt-get upgrade
```

Next, start your virtual environment.  Mine is called "test" so I start with:

```
source test/bin/activate
```

Next, install tensorflow:

```
pip install --upgrade tensorflow
```

Test your installation with the following code:

In [4]:
import tensorflow as tf
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [5]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


## Computational Graphs and Sessions

OK, now we have tf installed we can try to do something interesting with it.  But first we need to understand a little about the execution model and what happens internally with tensorflow.

Tensorflow uses graphs to define functions to execute.  Each constant, variable, and expression is  node of the graph.  This provides a sometimes complex but very flexible method for defining and executing functions.  Some of the important benefits are:

* Graphs can be evaluated starting at any node.  Evaluating a node implie evaluating all previous nodes.
* Parallel computations are easier to figure out since dependent relationships are explicitly determined by the graph.

Below is a very simple example of a tesorflow computational graph:

In [6]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape  # m= number of samples, n = number of predictors
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [7]:
reset_graph()

# Question: what learning method is implemented below?
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [8]:
theta_value, n

(array([[ -3.74651413e+01],
        [  4.35734153e-01],
        [  9.33829229e-03],
        [ -1.06622010e-01],
        [  6.44106984e-01],
        [ -4.25131839e-06],
        [ -3.77322501e-03],
        [ -4.26648885e-01],
        [ -4.40514028e-01]], dtype=float32), 8)

In the above setup, we see that the computational graph is evaluated on an object called a "session".

Below we have an example of a neural network with a single hidden layer that solves a classifier for the iris data set.  This is using some high-level tools provided byt tensorflow.

In [16]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


# import some data to play with
iris = load_iris()
#X = iris.data[:, :2]  # we only take the first two features.
X = iris.data  # we only take the first two features.
Y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

X_train.shape, y_train.shape

((120, 4), (120,))

In [22]:
reset_graph()

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
net = tf.contrib.learn.DNNClassifier(hidden_units=[6,6], n_classes=3, feature_columns=feature_cols)
net = tf.contrib.learn.SKCompat(net)
net.fit(X_train, y_train, batch_size=20, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_evaluation_master': '', '_session_config': None, '_tf_random_seed': None, '_num_worker_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_master': '', '_environment': 'local', '_task_type': None, '_is_chief': True, '_save_summary_steps': 100, '_model_dir': '/tmp/tmp8sk99a_j', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f5834d278>, '_task_id': 0}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp8sk99a_j/model.ckpt.
INFO:tensorflow:loss = 7.17847, step = 1
INFO:tensorflow:global_step/sec: 706.48
INFO:tensorflow:loss = 0.279563, step = 101 (0.147 sec)
INFO:tensorflow:global_step/sec: 544.314
INFO:tensorflow:loss = 0

SKCompat()

In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_predict = net.predict(X_test)
confusion_matrix(y_test, y_predict['classes'])

INFO:tensorflow:Restoring parameters from /tmp/tmpzerozvaq/model.ckpt-1000


array([[11,  0,  0],
       [ 0, 12,  1],
       [ 0,  0,  6]])

In [25]:
print(classification_report(y_test, y_predict['classes']))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        11
          1       1.00      0.92      0.96        13
          2       0.86      1.00      0.92         6

avg / total       0.97      0.97      0.97        30

